# TCGA 

## Initial Setup

Connects to Google Drive
Installs and imports dependancies
Run whether starting or returning

In [ ]:
%ls
%cd CancerPrediction
from multiprocessing import Process
import numpy as np
from numpy import asarray
import os
import pandas as pd
import shutil
from sys import platform
import glob
from matplotlib import pyplot as plt
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator
from PIL import Image
from matplotlib import pyplot as plt
import random 
from PIL import Image
plt.style.use("ggplot")
import openslide
import gc
# import sklearn
from sklearn.model_selection import train_test_split
import pickle
import cv2


In [ ]:
import os
process_count = len(os.sched_getaffinity(0))
print('Processors available: ', process_count)

## Download Selected Images

### Delete old - Only use if necessary

In [ ]:
# for img_dir in os.listdir('slides'): 
#   path = os.path.join('slides', img_dir)
#   if os.path.isdir(path) and 'ipynb' not in img_dir:
#     shutil.rmtree(path)

### Split manifest

In [ ]:
manifest_filename = 'slides/gdc_manifest_Tissue_Slides_non_annotated.txt'
manifest_filename_diag = 'slides/gdc_manifest_Tissue_Slides_diagnostic.txt'
df_manifest = pd.read_csv(manifest_filename, sep='\t')
df_manifest_diag = pd.read_csv(manifest_filename_diag, sep='\t')
df_manifest_all = pd.concat([df_manifest, df_manifest_diag]).drop_duplicates().reset_index(drop=True)
print(len(df_manifest_all))
list_df = np.array_split(df_manifest, 5)
df_manifest_slice = list_df[0]
df_manifest_slice.to_csv('slides/gdc_manifest_slice.txt', index=False, sep='\t')
print(len(df_manifest_slice))
df_manifest_slice.head()

### Download

In [ ]:
%cd slides
%ls
!chmod +x gdc-client
!./gdc-client download -m gdc_manifest_slice.txt
%cd -

## Create Image DataFrame

In [ ]:
# manifest_filename = 'slides/gdc_manifest_Tissue_Slides.txt'
df_manifest = df_manifest_slice
clinic_filename = 'CombinedClinicalManifestData.csv'
df_clinic = pd.read_csv(clinic_filename)
df_datalabels = df_clinic.merge(df_manifest, on='filename', how='inner', suffixes=('', '_y'))

df_datalabels.drop(df_datalabels.filter(regex='_y$').columns, axis=1, inplace=True)
df_datalabels['Path'] = df_datalabels['id'].apply(lambda x: '/slides/' + x + '/tiles/')
df_datalabels['ER_Label'] = df_datalabels['brca_shared-er_status_by_ihc'].apply(lambda x: 1 if x == 'Positive' else (0 if x == 'Negative' else 2))
df_datalabels['PR_Label'] = df_datalabels['brca_shared-pr_status_by_ihc'].apply(lambda x: 1 if x == 'Positive' else (0 if x == 'Negative' else 2))
df_datalabels['HER_Label'] = df_datalabels['brca_shared-her2_status_by_ihc'].apply(lambda x: 1 if x == 'Positive' else (0 if x == 'Negative' else 2))
df_datalabels['NNN_label'] = df_datalabels[['ER_Label', 'PR_Label', 'HER_Label']].apply(lambda x: 1 if (x.ER_Label == 0 and x.PR_Label == 0 and x.HER_Label == 0) else 0, axis=1)
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('md5')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('size')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('state')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('bcr_patient_barcode')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('shared-gender')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('clin_shared-vital_status')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('clin_shared-race_list')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('shared-tissue_source_site')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('shared-patient_id')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('shared-bcr_patient_uuid')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('clin_shared-patient_consent_status')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('clin_shared-icd_o_3_site')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('clin_shared-icd_o_3_histology')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('clin_shared-icd_10')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('clin_shared-initial_pathologic_dx_days_to')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('shared_stage-stage_event')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('clin_shared-form_completion_day')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('clin_shared-form_completion_month')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('clin_shared-form_completion_year')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('images')]
print(len(df_datalabels))
df_datalabels.to_csv('slides/model_datalabels.csv', index=False)

df_datalabels.head()

## Initialize files if continuing work after hiatus

In [ ]:
df_datalabels = pd.read_csv('slides/model_datalabels.csv')
# manifest_filename = 'slides/gdc_manifest_Tissue_Slides.txt'
# df_manifest = pd.read_csv(manifest_filename, sep='\t')
# clinic_filename = 'CombinedClinicalManifestData.csv'
# df_clinic = pd.read_csv(clinic_filename)
# images_to_process = pd.read_csv('slides/images_to_process.csv')
# print(len(images_to_process))
# images_to_process.head()
# # df_test_idea = images_to_process
# # df_test_idea

## Process the slides into tiles


#### Preprocessing Definitions

In [ ]:
def norm_HnE(img, Io=240, alpha=1, beta=0.15):
    # Step 1: Convert RGB to OD ###################
    # reference H&E OD matrix.
    # Can be updated if you know the best values for your image.
    # Otherwise use the following default values.
    # Read the above referenced papers on this topic.
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
    # reference maximum stain concentrations for H&E
    maxCRef = np.array([1.9705, 1.0308])

    # extract the height, width and num of channels of image
    h, w, c = img.shape

    # reshape image to multiple rows and 3 columns.
    # Num of rows depends on the image size (wxh)
    img = img.reshape((-1, 3))

    # calculate optical density
    # OD = −log10(I)
    # OD = -np.log10(img+0.004)  #Use this when reading images with skimage
    # Adding 0.004 just to avoid log of zero.

    
    OD = -np.log10((img.astype(float) + 1) / Io)  # Use this for opencv imread
    # Add 1 in case any pixels in the image have a value of 0 (log 0 is indeterminate)

    # Step 2: Remove data with OD intensity less than β ############
    # remove transparent pixels (clear region with no tissue)
    ODhat = OD[~np.any(OD < beta, axis=1)]  # Returns an array where OD values are above beta
    # Check by printing ODhat.min()

    # Step 3: Calculate SVD on the OD tuples ######################
    # Estimate covariance matrix of ODhat (transposed)
    # and then compute eigen values & eigenvectors.
    eigvals, eigvecs = np.linalg.eigh(np.cov(ODhat.T))

    # Step 4: Create plane from the SVD directions with two largest values ######
    # project on the plane spanned by the eigenvectors corresponding to the two
    # largest eigenvalues
    That = ODhat.dot(eigvecs[:, 1:3])  # Dot product

    # Step 5: Project data onto the plane, and normalize to unit length ###########
    # Step 6: Calculate angle of each point wrt the first SVD direction ########
    # find the min and max vectors and project back to OD space
    phi = np.arctan2(That[:, 1], That[:, 0])

    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100 - alpha)

    vMin = eigvecs[:, 1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
    vMax = eigvecs[:, 1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)

    # a heuristic to make the vector corresponding to hematoxylin first and the
    # one corresponding to eosin second
    if vMin[0] > vMax[0]:
        HE = np.array((vMin[:, 0], vMax[:, 0])).T

    else:
        HE = np.array((vMax[:, 0], vMin[:, 0])).T

    # rows correspond to channels (RGB), columns to OD values
    Y = np.reshape(OD, (-1, 3)).T

    # determine concentrations of the individual stains
    C = np.linalg.lstsq(HE, Y, rcond=None)[0]

    # normalize stain concentrations
    maxC = np.array([np.percentile(C[0, :], 99), np.percentile(C[1, :], 99)])
    tmp = np.divide(maxC, maxCRef)
    C2 = np.divide(C, tmp[:, np.newaxis])

    # Step 8: Convert extreme values back to OD space
    # recreate the normalized image using reference mixing matrix

    Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
    Inorm[Inorm > 255] = 254
    Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)

    # Separating H and E components

    H = np.multiply(Io, np.exp(np.expand_dims(-HERef[:, 0], axis=1).dot(np.expand_dims(C2[0, :], axis=0))))
    H[H > 255] = 254
    H = np.reshape(H.T, (h, w, 3)).astype(np.uint8)

    E = np.multiply(Io, np.exp(np.expand_dims(-HERef[:, 1], axis=1).dot(np.expand_dims(C2[1, :], axis=0))))
    E[E > 255] = 254
    E = np.reshape(E.T, (h, w, 3)).astype(np.uint8)

    return Inorm, H, E

In [ ]:
def is_blurry(image, threshold):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calculate the Laplacian of the image
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    # print(f'Blurriness: {laplacian_var}')
    # Check if the variance is below the threshold
    return laplacian_var < threshold


def save_loop(row1, col1, directory, tiles, imagename, levelnum):
    # df = pd.DataFrame(columns=['filename', 'ER_Label'])
    for row in row1:
        for col in col1:
            tile_name = os.path.join(directory, imagename + '_%d_%d' % (col, row))
            # print("Now saving tile with title: ", tile_name)
            temp_tile = tiles.get_tile(levelnum, (col, row))
            temp_tile_rgb = temp_tile.convert('RGB')
            temp_tile_np = np.array(temp_tile_rgb)
            #################################################
            # CHANGE MADE - PLEASE VERIFY
            if temp_tile_np.mean() < 180 and np.std(np.mean(temp_tile_np, axis=(0, 1))) > 5 and temp_tile_np.shape == (256, 256, 3) and not is_blurry(temp_tile_np, 250):
                # print("****Good tile number:", tile_name)
                plt.imsave(tile_name + ".png", temp_tile_np)
    # del temp_tile_np, temp_tile_rgb, temp_tile
    # gc.collect()
    # norm_img, H_img, E_img = norm_HnE(temp_tile_np, Io=240, alpha=1, beta=0.15)
    #
    # #Save the norm tile, H and E tiles
    # plt.imsave(tile_name + "_norm.png", norm_img)
    # plt.imsave(tile_name + "_H.png", H_img)
    # plt.imsave(tile_name + "_E.png", E_img)

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))


### Segment the tiles

In [ ]:
images_to_process = df_datalabels # If starting new
# images_to_process = pd.read_csv('slides/images_to_process.csv')  # If resuming from previous

images_to_process.head()

In [ ]:
import math
import time
# import pyvips
# manifest = df_datalabels

SLIDES_PATH = './slides'
num_to_process = len(images_to_process)
drop = []
for index, row in images_to_process.iterrows():
    tile_dir = f"{SLIDES_PATH}/{row.id}/tiles"
    process_count = math.isqrt(150)
    # process_count = math.isqrt(len(os.sched_getaffinity(0)))
    print('Processors available: ', process_count ** 2)
    print(row.id)


    # Load the slide file (svs) into an object.
    slide = open_slide(f"{SLIDES_PATH}/{row.id}/{row.filename}")

    slide_props = slide.properties

    try:
        objective = float(slide.properties[openslide.PROPERTY_NAME_OBJECTIVE_POWER])
         # get slide dimensions for the level 0 - max resolution level
        slide_dims = slide.dimensions
        # print(slide_dims)

        # Get slide dims at each level. Remember that whole slide images store information
        # as pyramid at various levels
        dims = slide.level_dimensions
        print(f'{objective}: {dims}')
        
        num_levels = len(dims)
        # By how much are levels downsampled from the original image?
        factors = slide.level_downsamples
        # Copy an image from a level
        level4_dim = dims[len(dims)-1]
        
        # Size of your output image
        # Remember that the output would be a RGBA image (Not, RGB)
        # Generate object for tiles using the DeepZoomGenerator
        tiles = DeepZoomGenerator(slide, tile_size=256, overlap=0, limit_bounds=False)
        if objective == 20:
            if len(dims) == 4:
                take = 1
            else:
                take = 1
            level_num = len(tiles.level_tiles) - take
            dim_lev = len(dims) - 1
        else:
            if len(dims) == 4:
                take = 2
            else:
                take = 2
            level_num = len(tiles.level_tiles) - take
            dim_lev = len(dims) - 1

        print(f'level_num: {level_num} out of {len(tiles.level_tiles) - 1}')
        level4_img = slide.read_region((0, 0), dim_lev, level4_dim)  # Pillow object, mode=RGBA
        
        
        # Convert the image to RGB
        level4_img_RGB = level4_img.convert('RGB')
        plt.imshow(level4_img_RGB)
        plt.show()
        
    
        # Generating tiles for deep learning training or other processing purposes
        # We can use read_region function and slide over the large image to extract tiles
        # but an easier approach would be to use DeepZoom based generator.
        # https://openslide.org/api/python/
        # Here, we have divided our svs into tiles of size 256 with no overlap.
        # The tiles object also contains data at many levels.
        if len(tiles.level_tiles) > level_num:
            if os.path.exists(tile_dir):
                shutil.rmtree(tile_dir)
            os.makedirs(tile_dir, exist_ok=True)
            level_dim = tiles.level_tiles[level_num]
            
            image_dim = tiles.level_dimensions[level_num]
            print(level_dim)
            print(f'Level {level_num}\'s dimensions: {image_dim}')
            # Preview a tile with tissue data
            while(True):
                midx = random.randint(0, level_dim[0])
                midy = random.randint(0, level_dim[1])
                try: 
                    tile_dims = tiles.get_tile_dimensions(level_num, (
                    midx, midy))  # Provide deep zoom level and address (column, row)
                    single_tile = tiles.get_tile(level_num, (
                        midx, midy))  # Provide deep zoom level and address (column, row)
                    single_tile_array = np.asarray(single_tile)
                    if single_tile_array.mean() < 180 and np.std(np.mean(single_tile_array, axis=(0, 1))) > 5 and single_tile_array.shape == (256, 256, 3) and not is_blurry(single_tile_array,100):
                        single_tile_RGB = single_tile.convert('RGB')
                        break
                except ValueError as ve:
                    pass
            print("Tile ", (midx, midy), "'s shape at level ", level_num, " is: ", tile_dims)
            plt.imshow(single_tile_RGB)
            plt.show()

          
            # Saving each tile to local directory
            cols, rows = tiles.level_tiles[level_num - 1]
            
            t_rows = tuple(split(range(rows), process_count))
            t_cols = tuple(split(range(cols), process_count))
            processes = []
            
            tic = time.time()
            # Using multiprocessing to speed up the process
            for i in range(process_count):
                for j in range(process_count):
                    p = Process(target=save_loop, args=(t_rows[i], t_cols[j], tile_dir, tiles,row.filename[:-4],
                                                        level_num - 1, ))
                    p.start()
                    processes.append(p)
            gc.collect()
            for p in processes:
                p.join()
            print(f'Time to process image is {time.time() - tic} seconds')
            files = os.listdir(tile_dir)
            print(f'Completed {tile_dir} {index + 1}/{num_to_process}')
            tile_count = len(os.listdir(tile_dir))
            print(f'Number of tiles: {tile_count}\n')
            images_to_process = images_to_process[images_to_process.id != row.id]
            images_to_process.to_csv('slides/images_to_process.csv', index=False)  
            ####### os.remove(f"{SLIDES_PATH}/{row.id}/{row.filename}")
        else:
            print('Image does not reach desireable level.\n')
            drop.append(row.id)
            shutil.rmtree(f"{SLIDES_PATH}/{row.id}")
    except KeyError as e:
        print(f'KeyError, on objective power. Removing slide')
        images_to_process = images_to_process[images_to_process.id != row.id]
        drop.append(row.id)
        print()
for item in drop:
    df_datalabels = df_datalabels[df_datalabels['id'] != item]
    images_to_process = images_to_process[images_to_process.id != row.id]
df_datalabels.to_csv('slides/model_datalabels.csv', index=False)

# # Memory cleanup
# del processes, process_count, t_rows, t_cols, single_tile, single_tile_RGB
# del level_dim, tile_dims, level4_img_np, level4_img, tiles
# del slide, dims

print(f"Complete. Removed {len(drop)} files due to insufficient level depth.")


## Dataframe for tiles and locations for tile dataframes

### Populate the WSI Tile Filepath Dataframes

In [ ]:
path = 'slides'
name = 'tile_data.csv'
df_datalabels.reset_index(drop=True)
for img_dir in os.listdir(path): 
  if os.path.isdir(os.path.join(path, img_dir)) and 'ipynb' not in img_dir:
    file_path = []
    for file in glob.glob(os.path.join(path, img_dir, 'tiles', '*.png')):
      file_path.append(file)
    df = pd.DataFrame()
    df['TilePath'] = file_path
    df.to_csv(os.path.join(path, img_dir, name), index=False)

df_datalabels['Tile_DF'] = df_datalabels.apply(lambda x: os.path.join(path, x.id, name), axis=1)
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('index')]
df_datalabels = df_datalabels.loc[:, ~df_datalabels.columns.str.contains('Unnamed')]
df_datalabels.to_csv('slides/model_datalabels_new.csv', index=False)

# Memory cleanup
del file_path, df

df_datalabels.head()


### Remove .svs file to save space

In [ ]:
SLIDES_PATH = './slides'
for index, row in df_datalabels.iterrows():
    os.remove(f"{SLIDES_PATH}/{row.id}/{row.filename}")
    # shutil.rmtree(f"{SLIDES_PATH}/{row.id}/logs")

### Drop insufficient tiled images

In [ ]:
# Drops images with insufficient number of tiles
n = 50 # Minimum number of tiles to keep WSIs
drop = []
for i in range(len(df_datalabels)):
  tile_count = len(pd.read_csv(df_datalabels.iloc[i]['Tile_DF']))
  if tile_count < n:
    drop.append(df_datalabels.iloc[i]['id'])
    print(f'{df_datalabels.iloc[i].id} with {tile_count} tiles')
  
df_reduced = df_datalabels[~df_datalabels.id.isin(drop)]
# df_reduced.to_csv('slides/reduce_model_labels.csv')
print(f'Number of usable samples: {len(df_reduced)}, number of samples removed: {len(drop)}')
del tile_count, drop

df_reduced.head()